In [4]:
import pandas as pd
data = pd.read_json('ft_data_3.json')
data

,prompt,content,label,token
0,请根据以下新闻文本，预测三一重工股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2018-10-10 09:00:00+08:00\n时间：2018-10-09 17...,极度负面,2291
1,请根据以下新闻文本，预测三一重工股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2018-10-24 09:00:00+08:00\n时间：2018-10-09 17...,极度负面,3099
2,请根据以下新闻文本，预测三一重工股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2018-12-07 09:00:00+08:00\n时间：2018-12-06 18...,负面,4949
3,请根据以下新闻文本，预测三一重工股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2018-12-24 09:00:00+08:00\n时间：2018-12-06 18...,负面,6829
4,请根据以下新闻文本，预测三一重工股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2018-12-24 09:00:00+08:00\n时间：2018-12-06 18...,负面,8136
...,...,...,...,...
5082,请根据以下新闻文本，预测中国建筑股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2023-03-31 09:00:00+08:00\n时间：2023-03-29 19...,正面,4210
5083,请根据以下新闻文本，预测中国建筑股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2023-03-31 09:00:00+08:00\n时间：2023-03-30 18...,正面,6582
5084,请根据以下新闻文本，预测中国建筑股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2023-03-31 09:00:00+08:00\n时间：2023-03-30 18...,正面,7217
5085,请根据以下新闻文本，预测中国建筑股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2023-03-31 09:00:00+08:00\n时间：2023-03-30 18...,正面,7687


In [5]:
data['label'].value_counts()

label
中性      1031
极度负面    1027
极度正面    1018
负面      1013
正面       998
Name: count, dtype: int64

In [6]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import json

# device = "cuda" if torch.cuda.is_available() else "cpu"


def load_model_and_tokenizer(model_name):
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto",  trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    return model, tokenizer


def predict(model, tokenizer, prompt):
    messages = [
        {"role": "system", "content": "你需要帮助我判断股票的对数收益率情况"},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt")
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        temperature=0.3,
    )
    response_tokens = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(response_tokens, skip_special_tokens=True)[0]
    return response


def evaluate_accuracy(model, tokenizer, test_data):
    results = []
    correct_predictions = 0
    for _, item in tqdm(test_data.iterrows(), total=test_data.shape[0]):
        predicted_response = predict(model, tokenizer, item['prompt'])
        expected_response = item['expected']
        results.append({
            "prompt": item['prompt'],
            "expected": expected_response,
            "predicted": predicted_response
        })
        if expected_response in predicted_response:
            correct_predictions += 1
    accuracy = correct_predictions / len(test_data)

    with open('test_dataset/evaluation_results_v2.json', 'w') as f:
        json.dump(results, f, indent=4, ensure_ascii=False)

    return accuracy


test_data = pd.read_json('test_dataset/results.json')[:20]
model_path = "../exp_model/v2"
# model_path = "/home/zhangmin/.cache/modelscope/hub/qwen/Qwen-7B-Chat"
model, tokenizer = load_model_and_tokenizer(model_path)

accuracy = evaluate_accuracy(model, tokenizer, test_data)
print("Accuracy:", accuracy)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]/home/zhangmin/anaconda3/envs/pytorch-2.0/lib/python3.8/site-packages/transformers/generation/utils.py:1465: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
 20%|██        | 4/20 [00:10<00:47,  3.00s/it]